In [36]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import cv2
import csv
import json
import os.path
import pickle
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Flatten, Lambda
from keras.optimizers import Adam
import tensorflow as tf
%matplotlib inline

In [38]:
# Import data
csv_file = 'driving_log.csv'
df = pd.read_csv(csv_file, header=None)
df.columns = ['center','left','right','steering','throttle','brake','speed']
df = df[(df.brake==0.0)&(df.speed>0.0)]
df.drop(['left','right','throttle','brake'], axis = 1)
image_path = np.copy(df['center'])

In [40]:
# Get images

width = 200
height = 66
depth = 3

def load_image(image_path):
    X_train = np.zeros(shape=[len(image_path), height, width, depth])
    for i, path in enumerate(image_path):
        image = mpimg.imread(path)
        shape = image.shape
        image = image[int(image.shape[0]/3):image.shape[0], :]
        image = cv2.resize(image, (width, height))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        X_train[i, :, :, :] = image
    return X_train
X_train = load_image(image_path)


In [43]:
# Get labels
Y_train = np.copy(df['steering'])
Y_train = Y_train.astype(np.float32)

In [ ]:
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5))
model.add(Convolution2D(5, 5, 24, subsample=(4, 4), border_mode="same"))
